# Workbook 3 Lasso
After selecting my own feature, I then used Lasso to see what features it selected and compared the scores and the coefficients

In [84]:
import numpy as np
import pandas as pd
import patsy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.feature_selection import SelectKBest

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [85]:
train = pd.read_csv('./train.csv')

In [86]:
test = pd.read_csv('./test.csv')

In [87]:
y = train['SalePrice']
X = train.drop(['SalePrice'], axis = "columns")

In [88]:
test_id = test['Id']

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [90]:
def clean_me(df):
    #Replaces Null values outside of object columns 
    for i in range(len(df.columns)): 
        if [df.iloc[: ,i].dtypes!=object] == True:
            mean = df.iloc[: ,i].mean()
            df.iloc[: ,i].fillna(mean, inplace=True)
        else:
            pass
        #Replaces other null values with AANone
    str_cols = df.columns[df.dtypes==object]
    df[str_cols] = df[str_cols].fillna('AANone')
    df.fillna(0,inplace=True)
        #Change selected columns from Int to float
    for col in ['Low Qual Fin SF', 'Gr Liv Area', 'Lot Area', '1st Flr SF', '2nd Flr SF',
                'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val']:
                df[col] = df[col].astype('float64')
        #Makes remaining int colomns objects 
    int_cols = df.columns[df.dtypes=='int64']
    df[int_cols] = df[int_cols].astype('object')
    df = df.drop(['Id', 'PID'], axis=1)
    return df

In [91]:
X_train = clean_me(X_train)

C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [92]:
X_test = clean_me(X_test)

C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

In [93]:
test = clean_me(test)

**Cleaning Done**

Used lasso with all the variables to pair down the coefficience

In [94]:
def get_dummied(train, test):
    list = []
    
    for i in train.columns:
        if train[i].dtype == object:
            list.append(i)
            #puts all object names in list
    full_data = pd.concat([train, test], axis=0)
    #puts training data and test data in one dataframe
    full_data = pd.get_dummies(full_data, columns=list, drop_first=True)
    #applies get dummy to whole list
    X_dummied = full_data[:len(train)]
    test_dummied = full_data.iloc[-len(test):]
    #splits full data back into 2 data frames
    
    return X_dummied, test_dummied

In [95]:
train_dum, test_dum = get_dummied(X_train, X_test)

In [98]:
 ss = StandardScaler()

In [99]:
train_scaled = ss.fit_transform(train_dum)

**NOTE**

should be transforming on fit done on train. Will be fixed in future updates

In [100]:
test_scaled = ss.fit_transform(test_dum)

In [101]:
#l_alphas = np.arange(0.001, 0.2, 0.0001) 
lasso_m = LassoCV( cv=3)
lasso_m =lasso_m.fit(train_scaled, y_train)
lasso_optimal_alpha = lasso_m.alpha_
print(lasso_optimal_alpha,':is the optimal alpha')

print(lasso_m.score(test_scaled, y_test))

999.8996577022181 :is the optimal alpha
0.8939807348036363


In [102]:
lasso_m.coef_

array([ 0.00000000e+00,  2.05441135e+03,  3.75119501e+03,  4.30450181e+03,
        1.07778534e+03, -0.00000000e+00,  3.66499855e+03,  3.37510560e+03,
        0.00000000e+00, -2.50861650e+02,  1.94688100e+04,  1.61649389e+03,
       -0.00000000e+00, -0.00000000e+00,  6.86305208e+03,  0.00000000e+00,
        1.93018432e+03,  2.58766796e+02, -0.00000000e+00,  3.25853353e+02,
        3.12315243e+03,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -1.93069011e+02, -1.38506036e+03, -2.54241246e+01,
       -0.00000000e+00, -3.44304270e+02, -5.30382753e+02,  1.07272880e+02,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.35672836e+03,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  8.33587465e+02,
       -4.50393911e+03, -0.00000000e+00,  1.92141088e+02,  0.00000000e+00,
       -0.00000000e+00, -

In [108]:
train_dum, pred_dum = get_dummied(X_train, test)

In [109]:
train_scaled = ss.fit_transform(train_dum)

In [110]:
pred_scaled = ss.fit_transform(pred_dum)

In [111]:
lasso_m = LassoCV( cv=3)
lasso_m =lasso_m.fit(train_scaled, y_train)
lasso_optimal_alpha = lasso_m.alpha_

In [112]:
y_hat = lasso_m.predict(pred_scaled)

In [114]:
#scale_trial_2 = pd.DataFrame(y_hat, index = test_id)
#scale_trial_2.columns = ['SalePrice']
#scale_trial_2.head()

,SalePrice
Id,
2658,125336.343646
2718,166959.196746
2414,210848.683388
1989,123624.395455
625,189013.812195


In [115]:
#scale_trial_2.to_csv("./scale_trial_2.csv")

Used good coefficients and used poly features and lassoed again. Did not work

In [ ]:
coefs = pd.DataFrame([lasso_m.coef_], columns= train_dum.columns)
coefs = coefs.T

In [ ]:
mask = coefs[0] > 0
good = coefs[mask]
good = good.T

In [229]:
pd.options.display.max_rows = 999

In [231]:
pd.options.display.max_rows

999

Coefficient List (most important features)

In [232]:
good.T

,0
Lot Area,2054.413702
Mas Vnr Area,3751.197423
BsmtFin SF 1,4304.499654
BsmtFin SF 2,1077.786296
Total Bsmt SF,3664.996092
1st Flr SF,3375.113854
Gr Liv Area,19468.799724
Bsmt Full Bath,1616.491659
Garage Cars,6863.049064
Wood Deck SF,1930.182800


In [152]:
good.columns

Index(['Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
       'Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Garage Cars', 'Wood Deck SF', 'Open Porch SF', '3Ssn Porch',
       'Screen Porch', 'MS Zoning_FV', 'Lot Shape_IR2', 'Land Contour_HLS',
       'Lot Config_CulDSac', 'Neighborhood_ClearCr', 'Neighborhood_Crawfor',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr', 'Neighborhood_Veenker', 'Condition 1_Norm',
       'Condition 1_PosA', 'Condition 1_PosN', 'Overall Qual_7',
       'Overall Qual_8', 'Overall Qual_9', 'Overall Qual_10', 'Overall Cond_7',
       'Overall Cond_8', 'Overall Cond_9', 'Year Built_1892',
       'Year Built_1932', 'Year Built_1991', 'Year Built_1998',
       'Year Built_2006', 'Year Built_2008', 'Year Built_2009',
       'Year Remod/Add_1996', 'Year Remod/Add_2001', 'Year Remod/Add_2006',
       'Year Remod/Add_2007', 'Year Remod/Add_2008', 'Year Remod/Add_2010',

In [185]:
lasso_col = ['Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2',
       'Total Bsmt SF', '1st Flr SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Garage Cars', 'Wood Deck SF', 'Open Porch SF', '3Ssn Porch',
       'Screen Porch', 'MS Zoning_FV', 'Lot Shape_IR2', 'Land Contour_HLS',
       'Lot Config_CulDSac', 'Neighborhood_ClearCr', 'Neighborhood_Crawfor',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_Somerst',
       'Neighborhood_StoneBr', 'Neighborhood_Veenker', 'Condition 1_Norm',
       'Condition 1_PosA', 'Condition 1_PosN', 'Overall Qual_7',
       'Overall Qual_8', 'Overall Qual_9', 'Overall Qual_10', 'Overall Cond_7',
       'Overall Cond_8', 'Overall Cond_9', 'Year Built_1892',
       'Year Built_1932', 'Year Built_1991', 'Year Built_1998',
       'Year Built_2006', 'Year Built_2008', 'Year Built_2009',
       'Year Remod/Add_1996', 'Year Remod/Add_2001', 'Year Remod/Add_2006',
       'Year Remod/Add_2007', 'Year Remod/Add_2008', 'Year Remod/Add_2010',
       'Roof Matl_Membran', 'Exterior 1st_BrkFace', 'Exterior 1st_CemntBd',
       'Exterior 1st_VinylSd', 'Exterior 2nd_VinylSd', 'Mas Vnr Type_Stone',
       'Foundation_PConc', 'Bsmt Qual_Ex', 'Bsmt Qual_Gd', 'Bsmt Exposure_Gd',
       'BsmtFin Type 1_ALQ', 'BsmtFin Type 1_GLQ', 'Full Bath_3',
       'Half Bath_1', 'Bedroom AbvGr_4', 'Kitchen AbvGr_1', 'TotRms AbvGrd_8',
       'TotRms AbvGrd_9', 'TotRms AbvGrd_10', 'TotRms AbvGrd_11',
       'Functional_Typ', 'Fireplaces_1', 'Fireplaces_2', 'Fireplaces_4',
       'Fireplace Qu_Gd', 'Fireplace Qu_TA', 'Garage Type_Attchd',
       'Garage Finish_Fin', 'Garage Qual_Ex', 'Garage Qual_Gd',
       'Paved Drive_Y', 'Sale Type_Con', 'Sale Type_New', 'Sale Type_Oth']

# Attempt made on second Lasso,
Tried to use polyfeatures on top 50 and lasso 

In [213]:
train_dum, test_dum = get_dummied(X_train, X_test)

In [214]:
train_good = train_dum[lasso_col]

In [215]:
test_good = test_dum[lasso_col]

In [216]:
train_scaled_good = ss.fit_transform(train_good)

In [217]:
test_scaled_good = ss.fit_transform(test_good)

In [218]:
poly = PolynomialFeatures(include_bias = True)

In [219]:
train_scaled_poly_good = poly.fit_transform(train_scaled_good)

In [220]:
test_scaled_poly_good = poly.fit_transform(test_scaled_good)

In [221]:
lasso_m = LassoCV( cv=3)
lasso_m =lasso_m.fit(train_scaled_poly_good, y_train)
lasso_optimal_alpha = lasso_m.alpha_
print(lasso_optimal_alpha,':is the optimal alpha')

print(lasso_m.score(test_scaled_poly_good, y_test))

C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


444.74925992073867 :is the optimal alpha
0.874628278916928


C:\Users\jmvail\Anaconda3\envs\dsi\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [222]:
lasso_m.coef_

array([   0.        , 5176.97547048, 1312.66202262, ...,  185.0038248 ,
         -0.        ,   95.14818582])

In [183]:
good

,Lot Area,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Total Bsmt SF,1st Flr SF,Gr Liv Area,Bsmt Full Bath,Garage Cars,Wood Deck SF,...,Fireplace Qu_Gd,Fireplace Qu_TA,Garage Type_Attchd,Garage Finish_Fin,Garage Qual_Ex,Garage Qual_Gd,Paved Drive_Y,Sale Type_Con,Sale Type_New,Sale Type_Oth
0,2054.413702,3751.197423,4304.499654,1077.786296,3664.996092,3375.113854,19468.799724,1616.491659,6863.049064,1930.1828,...,924.650215,466.059409,925.289294,1843.391485,30.901618,3761.414476,606.3208,371.984521,2408.140801,468.520118
